<a href="https://colab.research.google.com/github/sundaybest3/NounSmart_countability/blob/main/countability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio pandas



In [ ]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart_countability/refs/heads/main/nouns_CE_visang.csv"
df = pd.read_csv(csv_url)

# Inspect the columns
expected_columns = ["Page", "Word", "Countability"]
missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"The following required columns are missing in the CSV: {', '.join(missing_columns)}")

# Initialize game state
game_state = {
    "nickname": None,
    "page": None,
    "current_noun": None,
    "remaining_nouns": None,
    "total_nouns": 0,
    "score": 0,
    "trials": 0,
}

# Function to filter nouns by page and prepare the list of nouns
def initialize_nouns(page):
    filtered_df = df[df["Page"] == page]
    if filtered_df.empty:
        return "No nouns available for this page. Please select a different page.", ""
    game_state["remaining_nouns"] = filtered_df.sample(frac=1).to_dict(orient="records")
    game_state["total_nouns"] = len(game_state["remaining_nouns"])
    game_state["score"] = 0
    game_state["trials"] = 0
    return show_random_noun(), "Ready to start!"

# Function to reset the game state when a new page is chosen
def reset_game_state():
    game_state["current_noun"] = None
    game_state["remaining_nouns"] = None
    game_state["total_nouns"] = 0
    game_state["score"] = 0
    game_state["trials"] = 0

# Function to show a random noun
def show_random_noun():
    if not game_state["remaining_nouns"]:
        return "No more nouns available. Click 'Submit Answer' to see your final feedback."

    game_state["current_noun"] = game_state["remaining_nouns"].pop()
    return game_state["current_noun"]["Word"]

# Function to check user's answer
def check_answer(user_choice):
    if not game_state.get("current_noun"):
        return "Please click 'Show the Noun' first."

    correct_answer = game_state["current_noun"]["Countability"].strip().lower()
    game_state["trials"] += 1

    if user_choice.lower() == correct_answer:
        game_state["score"] += 1
        feedback = f"✅ Correct! {game_state['current_noun']['Word']} is {correct_answer}. Click 'Show the Noun' to continue."
    else:
        game_state["remaining_nouns"].insert(0, game_state["current_noun"])
        feedback = f"❌ Incorrect. {game_state['current_noun']['Word']} is actually {correct_answer}. It will appear again. Click 'Show the Noun' to continue."

    # Check if all nouns are processed correctly
    if game_state["score"] == game_state["total_nouns"]:
        feedback += f"\n🎉 Great job, {game_state['nickname']}! All nouns have been answered correctly. Final Score: {game_state['score']} / {game_state['trials']} Choose another page for more practice."
    else:
        feedback += f"\nCurrent Score: {game_state['score']} / {game_state['trials']}"

    return feedback

# Function to get page summary with noun count
def get_page_summary():
    summary = df.groupby("Page").size().reset_index(name="Total")
    summary["Display"] = summary["Page"].astype(str) + " (" + summary["Total"].astype(str) + " nouns)"
    return summary[["Page", "Display"]]

# Gradio interface
def main_interface(nickname, page, user_choice=None):
    if not nickname:
        return "⛔ Enter a nickname before you start.", ""

    if page != game_state["page"]:
        reset_game_state()

    game_state["nickname"] = nickname
    game_state["page"] = page

    if game_state["remaining_nouns"] is None:
        noun, feedback = initialize_nouns(page)
        return noun, feedback

    if user_choice is None:
        return show_random_noun(), "Ready to continue!"

    return game_state["current_noun"]["Word"], check_answer(user_choice)

# Prepare page choices
page_summary = get_page_summary()
page_choices = page_summary.set_index("Page")["Display"].to_dict()

# Gradio app components
with gr.Blocks() as app:
    gr.Markdown("# NounSmart: Countability\n### Find the textbook page you selected and find out whether the noun is countable or not.")
    nickname = gr.Textbox(label="Enter your nickname:")
    page = gr.Dropdown(choices=list(page_choices.values()), label="Select a page:")
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Is this noun countable or uncountable?:", interactive=False)
    user_choice = gr.Radio(choices=["Countable", "Uncountable"], label="Your answer:")
    submit_button = gr.Button("Submit Answer")
    feedback_display = gr.Textbox(label="Feedback and Score:", interactive=False)

    # Map display back to page for processing
    def map_page_choice(display_choice):
        return next(page for page, display in page_choices.items() if display == display_choice)

    show_button.click(fn=lambda n, p: main_interface(n, map_page_choice(p)),
                      inputs=[nickname, page],
                      outputs=[noun_display, feedback_display])
    submit_button.click(fn=lambda n, p, c: main_interface(n, map_page_choice(p), c),
                        inputs=[nickname, page, user_choice],
                        outputs=[noun_display, feedback_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://115d6f243db93661e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
